In [1]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F

import pytorch_forecasting
from pytorch_forecasting.models import TemporalFusionTransformer

import ray

from utils import *
import pickle
import os
from pathlib import Path
from collections import defaultdict


/Users/bigc/opt/anaconda3/envs/ML-Env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pred_network import *

device = T.device("cuda:0" if T.cuda.is_available() else "cpu")

encoder_args = {'batch_size': 3, 'window_size': 120,
                    'features': 135}
decoder_args = {}

model = VGG16_AE(encoder_args, decoder_args, device)

Reversing module: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False): : 0it [00:00, ?it/s]

ModuleList(
  (0): MaxUnpool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
)
Encoder(
  (encoder): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15)

In [4]:
from utils import StockData

dataset = StockData('/Users/bigc/Documents/Code - Offline/dataset.pickle', 128, device)

In [5]:
print(dataset[0].shape)

torch.Size([128, 462])


In [7]:
from utils import *

data = load_dataset('/Users/bigc/Documents/Code - Offline/dataset.pickle')
print(data['ZTS'])

           open        high         low       close    adjclose     volume  \
0    178.960007  179.679993  176.809998  176.970001  176.271286  2449100.0   
1    177.929993  180.229996  177.059998  178.470001  177.765366  1621200.0   
2    179.080002  182.220001  178.179993  181.309998  180.594147  2339000.0   
3    180.000000  181.860001  176.789993  177.250000  176.550171  2251800.0   
4    176.389999  177.250000  171.550003  174.669998  173.980362  3513000.0   
..          ...         ...         ...         ...         ...        ...   
457  152.110001  153.339996  149.839996  150.779999  150.779999  1899000.0   
458  152.110001  153.339996  149.839996  150.779999  150.779999  1899000.0   
459  152.110001  153.339996  149.839996  150.779999  150.779999  1899000.0   
460  152.110001  153.339996  149.839996  150.779999  150.779999  1899000.0   
461  152.110001  153.339996  149.839996  150.779999  150.779999  1899000.0   

       volume_adi  volume_obv  volume_cmf     volume_fi  ...   